In [ ]:
import numpy as np
import pandas as pd

Загрузим данные и удалим столбец unnamed (индексы, образовавшиеся после загрузки) и артикул - он нам не нужен для обучения моделей

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/tisheedesh/Project/main/Data/data_edaed.csv').drop(['Unnamed: 0', 'Артикул'], axis=1)

Разобъём номер страницы на пять интервалов, предполагая, что на первых страницах самые популярные кольца, а на последних - самые непопулярные. Группы будут такими: Бестселлеры - Популярные - Средние - Непопулярные - Отвергнутые. Группы будем делить равномерно по номерам страниц. (мы это делали ранее, но сделаем еще раз, так как файл подгружен изначальный)

In [ ]:
df['Популярность'] = 0
df.loc[df['Номер страницы'] < 146 // 5, 'Популярность'] = 'Бестселлеры'
df.loc[(df['Номер страницы'] >= 146 // 5) & (df['Номер страницы'] < 146 // 5 * 2), 'Популярность'] = 'Популярные'
df.loc[(df['Номер страницы'] >= 146 // 5 * 2) & (df['Номер страницы'] < 146 // 5 * 3), 'Популярность'] = 'Средние'
df.loc[(df['Номер страницы'] >= 146 // 5 * 3) & (df['Номер страницы'] < 146 // 5 * 4), 'Популярность'] = 'Непопулярные'
df.loc[df['Номер страницы'] >= 146 // 5 * 4, 'Популярность'] = 'Отвергнутые'

In [ ]:
df.head(5)

,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,Количество,Цвет,Вес,Ширина кольца,Ширина,Высота,Номер страницы,Цена со скидкой,Цена без скидки,Популярность
0,Для женщин,1.35,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,1.0,Бесцветный,0.030,2.460293,7.272492,3.373794,1,35100,77990,Бестселлеры
1,Для женщин,0.90,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,23.0,Бесцветный,0.069,2.000000,2.000000,1.000000,1,7990,7990,Бестселлеры
2,Для женщин,1.31,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,57.0,Бесцветный,0.165,2.460293,7.272492,3.373794,1,56250,124990,Бестселлеры
3,Для женщин,0.99,Красное золото,585.0,Неизвестно,Фианит,Круг,1.0,Бесцветный,0.846,2.000000,5.000000,4.000000,1,6990,6990,Бестселлеры
4,Для женщин,0.97,Красное золото,585.0,Родирование,Фианит,Круг,5.0,Бесцветный,1.004,1.000000,3.000000,3.000000,1,6490,6490,Бестселлеры


Сделаем OneHotEncoding для качественных признаков:

In [ ]:
cat_features = ['Для кого', 'Тип металла', 'Покрытие', 'Тип вставки', 'Форма вставки', 'Цвет']

In [ ]:
cat_features.append('Популярность')
#добавляем признак Популярность в список категориальных

df_reg = pd.get_dummies(df.drop(['Номер страницы'], axis=1), columns=cat_features)
df_reg.head(5)

,Примерный вес,Проба,Количество,Вес,Ширина кольца,Ширина,Высота,Цена со скидкой,Цена без скидки,Для кого_Для детей,...,Цвет_Зелёный,Цвет_Неизвестно,Цвет_Розовый,Цвет_Синий,Цвет_Чёрный,Популярность_Бестселлеры,Популярность_Непопулярные,Популярность_Отвергнутые,Популярность_Популярные,Популярность_Средние
0,1.35,585.0,1.0,0.030,2.460293,7.272492,3.373794,35100,77990,False,...,False,False,False,False,False,True,False,False,False,False
1,0.90,585.0,23.0,0.069,2.000000,2.000000,1.000000,7990,7990,False,...,False,False,False,False,False,True,False,False,False,False
2,1.31,585.0,57.0,0.165,2.460293,7.272492,3.373794,56250,124990,False,...,False,False,False,False,False,True,False,False,False,False
3,0.99,585.0,1.0,0.846,2.000000,5.000000,4.000000,6990,6990,False,...,False,False,False,False,False,True,False,False,False,False
4,0.97,585.0,5.0,1.004,1.000000,3.000000,3.000000,6490,6490,False,...,False,False,False,False,False,True,False,False,False,False


# Модели

Начнём с регрессий:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Разделим выборку на обучающую и тестовую. Предсказывать будем цену со скидкой - как мы отмечали ранее в разделе EDA, она, по нашему мнению, точнее отражает стоимость, чем цена без скидки.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_reg.drop(['Цена со скидкой', 'Цена без скидки'], axis=1), df_reg['Цена со скидкой'], test_size=0.3, random_state=0)

Применим StandartScaler, чтобы нормировать признаки для возможности сопоставления их весов и лучшей обучаемости модели.

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

array([[ 0.04345424, -0.57320033, -0.45085137, ..., -0.50313876,
        -0.49580881, -0.51929999],
       [ 0.56658556, -0.57320033, -0.40296525, ..., -0.50313876,
        -0.49580881,  1.92566921],
       [-0.47175086,  1.50745567, -0.01987635, ..., -0.50313876,
        -0.49580881, -0.51929999],
       ...,
       [-0.82050508,  1.50745567, -0.25930691, ..., -0.50313876,
         2.01690647, -0.51929999],
       [ 0.32087236, -0.57320033, -0.40296525, ..., -0.50313876,
        -0.49580881, -0.51929999],
       [-0.41626723, -0.57320033, -0.40296525, ..., -0.50313876,
        -0.49580881, -0.51929999]])

Создадим микро функцию, которая будет выводить базовое описание полученной модели - ошибку RMSE, коэффициенты при признаках.

In [ ]:
def reg_description(y_test, X_test, reg):
    print('RMSE:', mean_squared_error(y_test, reg.predict(X_test)) ** 0.5)
    coefs = {'Признак': df_reg.drop(['Цена без скидки', 'Цена со скидкой'], axis=1).columns, 'Коэффициент': reg.coef_}
    print('Intercept:', reg.intercept_)
    display(pd.DataFrame(coefs).sort_values(by='Коэффициент'))

Наконец, обучим модель и применим функию reg_description для получения результатов:

In [ ]:
reg = LinearRegression().fit(X_train, y_train)
reg_description(y_test, X_test, reg)

RMSE: 69937.91825507036
Intercept: -193569.77755677182


,Признак,Коэффициент
18,Тип металла_Платина,-58529.532913
49,Цвет_Жёлтый,-57635.030655
11,Для кого_Для мужчин,-52154.789466
22,Тип металла_Чернёное серебро,-38718.703742
21,Тип металла_Сталь,-38327.547919
48,Цвет_Голубой,-36235.022988
36,Форма вставки_Багет,-34298.088083
34,Тип вставки_Фианит,-33515.756350
9,"Для кого_Для женщин, Для мужчин, Для детей",-26305.594445
44,Форма вставки_Октагон,-22183.007564


Получили что-то очень странное. RMSE в 70К рублей - большая ошибка, да и распределение весов у признаков не поддаётся логичному описанию - например, платина в металле кольца имеет самый низкий коэффициент: -58,5K рублей, что довольно странно, учитывая соотношение стоимостей платины и золота. Красное золото даёт больший импакт на цену, чем жёлтое, хотя процентное содержание чистого золота в красном золоте наименьшее (довольно большую часть занимает медь), из-за чего красное золото ценится меньше чем другие виды. В общем, странные коэффициенты.

Для сравнения попробуем посчитать ошибку наивного регрессора, который выдаёт среднее по y_train

In [ ]:
print('Naive RMSE:', mean_squared_error(y_test, np.full(shape=y_test.shape, fill_value=y_train.mean())) ** 0.5)

Naive RMSE: 87385.47500547033


Видим, что ошибка нашего регрессора лучше наивного всего лишь на 17К рублей, что довольно грустно.

Скорее всего это связано с большим числом выбросов, обнаруженных в EDA - много колец с экстремальными ценами или значениями признаков. Попробуем обучить регрессию с Huber Loss - функцией, которая позволяет "не уделять большое внимание" выбросам, считать их с меньшим весом в функции ошибки. Также Huber регрессор имеет регуляризацию, что позволит скорректировать веса на возможную мультиколлинеарность и переобучение.

In [ ]:
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import GridSearchCV

Будем подбирать два гиперпараметра - силу регуляризации alpha и порог отсечения выбросов epsilon. Будем выбирать их подряд, а не одновременно, потому что одновременно - очень долго. Начнём с подбора epsilon. Нам в этом поможет GridSearchCV - отбор параметров будет происходить сразу с кросс валидацией.

In [ ]:
e = np.linspace(1.0, 500, 50)
huber = HuberRegressor(max_iter = 5000, alpha=0.0)
grids = GridSearchCV(huber, {'epsilon': e})
grids.fit(X_train, y_train)
epsilon = grids.best_params_

In [ ]:
epsilon

{'epsilon': 337.06122448979596}

Теперь подберём альфу при выбранном эпсилоне

In [ ]:
a = np.linspace(0.0, 0.00001, 50)
huber = HuberRegressor(max_iter = 5000, epsilon=epsilon['epsilon'])
grids = GridSearchCV(huber, {'alpha': a})
grids.fit(X_train, y_train)
alpha = grids.best_params_

In [ ]:
alpha

{'alpha': 6.938775510204082e-06}

Сила регуляризации получилась крайне маленькой, что странно. Но ладно, обучим модель с найденными параметрами:

In [ ]:
huber_par = HuberRegressor(max_iter=5000, alpha=alpha['alpha'], epsilon=epsilon['epsilon'])
huber_par.fit(X_train, y_train)

reg_description(y_test, X_test, huber_par)

RMSE: 70194.32607832714
Intercept: -21126.72524987432


,Признак,Коэффициент
16,Тип металла_Комбинированное золото,-39019.954511
11,Для кого_Для мужчин,-39005.735481
48,Цвет_Голубой,-36201.239341
33,Тип вставки_Топаз,-33139.983998
34,Тип вставки_Фианит,-32072.273329
59,Популярность_Средние,-17509.703829
55,Популярность_Бестселлеры,-17185.070286
14,Тип металла_Жёлтое золото,-16534.437767
22,Тип металла_Чернёное серебро,-15249.928750
58,Популярность_Популярные,-14683.533597


Лучше не стало. Ошибка всё так же большая, даже больше, чем в обычной регрессии. При этом веса признаков скорректировались (как минимум, наличие бриллианта теперь считается весомым аргументом прибавки к цене). Попробуем ещё один регрессор - случайный лес.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

Также с помощью gridsearchcv подберём два гиперпараметра, но уже одновременно - максимальную глубину деревьев и минимальное разделение в листья.

In [ ]:
forest = RandomForestRegressor(random_state=0)

grids = GridSearchCV(forest, {'max_depth': list(map(int, np.linspace(10, 100, 15))), 'min_samples_split': list(map(int, np.linspace(2, 100, 15)))}, cv=3)
grids.fit(X_train, y_train)

params = grids.best_params_

In [ ]:
params

{'max_depth': 10, 'min_samples_split': 9}

Обучим лес с подобранными параметрами:

In [ ]:
forest = RandomForestRegressor(n_estimators=500, random_state=0, max_depth=params['max_depth'], min_samples_split=params['min_samples_split']).fit(X_train, y_train)

print('RMSE:', mean_squared_error(y_test, forest.predict(X_test)) ** 0.5)

RMSE: 24916.899666477246


Ошибка стала меньше в несколько раз, но всё ещё 25К рублей - слишком много для нашего диапазона стоимости. Но лучших результатов, определённо, не добиться.

Попробуем классифицировать кольца по популярности - признаку, введенному в начале этого ноутбука. Начнём с KNN:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

Заново определим категориальные фичи и сделаем OHE для них:

In [ ]:
cat_features = ['Для кого', 'Тип металла', 'Покрытие', 'Тип вставки', 'Форма вставки', 'Цвет']
df_clf = pd.get_dummies(df.drop(['Номер страницы'], axis=1), columns=cat_features)

Разделим выборку на обучающую и тестовую

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_clf.drop(['Популярность'], axis=1), df_clf['Популярность'], test_size=0.3, random_state=0)

Применим standartscaler, чтобы избежать проблем с разными масштабами расстояний:

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

array([[ 0.04345424, -0.57320033, -0.45085137, ..., -0.0409616 ,
        -0.02589336, -0.08417256],
       [ 0.56658556, -0.57320033, -0.40296525, ..., -0.0409616 ,
        -0.02589336, -0.08417256],
       [-0.47175086,  1.50745567, -0.01987635, ..., -0.0409616 ,
        -0.02589336, -0.08417256],
       ...,
       [-0.82050508,  1.50745567, -0.25930691, ..., -0.0409616 ,
        -0.02589336, -0.08417256],
       [ 0.32087236, -0.57320033, -0.40296525, ..., -0.0409616 ,
        -0.02589336, -0.08417256],
       [-0.41626723, -0.57320033, -0.40296525, ..., -0.0409616 ,
        -0.02589336, -0.08417256]])

Теперь с помощоью gridsearchcv подберем оптимальное число соседей:

In [ ]:
neigh = KNeighborsClassifier()

grids = GridSearchCV(neigh, {'n_neighbors' : list(map(int, np.linspace(2, 100, 50)))})
grids.fit(X_train, y_train)

n_neigh = grids.best_params_['n_neighbors']

In [ ]:
n_neigh

30

С найденным числом соседей обучим модель и проверим её на метриках Precision, Recall и F1-score (так как у нас multiclass классификация, для расчёта используем усреднение - а именно 'weighted')

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=n_neigh)
neigh.fit(X_train, y_train)

print('Test:')
print('Precision', precision_score(y_test, neigh.predict(X_test), average='weighted'))
print('Recall', recall_score(y_test, neigh.predict(X_test), average='weighted'))
print('F1', f1_score(y_test, neigh.predict(X_test), average='weighted'))

print('Train:')
print('Precision', precision_score(y_train, neigh.predict(X_train), average='weighted'))
print('Recall', recall_score(y_train, neigh.predict(X_train), average='weighted'))
print('F1', f1_score(y_train, neigh.predict(X_train), average='weighted'))

Test:
Precision 0.3122301672307218
Recall 0.32265625
F1 0.31407548537137214
Train:
Precision 0.37029308007615275
Recall 0.37889447236180906
F1 0.3697046936749343


Резуультаты, мягко говоря, отстойные. Попробуем Классифицировать случайным лесом:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Подбираем гиперпараметры (те же, что и в лесу-регрессоре)

In [ ]:
forest = RandomForestClassifier(random_state=0)

grids = GridSearchCV(forest, {'max_depth': list(map(int, np.linspace(10, 100, 15))), 'min_samples_split': list(map(int, np.linspace(2, 100, 15)))}, cv=3)
grids.fit(X_train, y_train)

params = grids.best_params_

In [ ]:
params

{'max_depth': 16, 'min_samples_split': 23}

Обучаем и тестируем модель:

In [ ]:
forest = RandomForestClassifier(n_estimators=500, random_state=0, max_depth=params['max_depth'], min_samples_split=params['min_samples_split']).fit(X_train, y_train)

print('Test:')
print('Precision', precision_score(y_test, forest.predict(X_test), average='weighted'))
print('Recall', recall_score(y_test, forest.predict(X_test), average='weighted'))
print('F1', f1_score(y_test, forest.predict(X_test), average='weighted'))

print('Train:')
print('Precision', precision_score(y_train, forest.predict(X_train), average='weighted'))
print('Recall', recall_score(y_train, forest.predict(X_train), average='weighted'))
print('F1', f1_score(y_train, forest.predict(X_train), average='weighted'))

Test:
Precision 0.43773631447740097
Recall 0.4453125
F1 0.43626906366431484
Train:
Precision 0.6572299110606911
Recall 0.6569514237855947
F1 0.6533229887133352


Вышло лучше, чем с KNN, но всё ещё проще подкинуть монетку и сказать, принадлежит ли объект к данному классу, или нет.

Таким образом у нас не получилось подобрать модель, способную достаточно хорошо предсказать цену или популярность кольца. Это может быть связанно как с особенностями ценообразования в ювелирном бизнесе (влияние ручной работы и др факторов), так и с тем, что мы убрали большое число признаков и объектов из выборки, лишив её, возможно, супер важных признаков, определяющих цену и популярность кольца. Выборка также характеризуется большим количеством выбросов: экстремальные значения были как в цене, так и в отдельных признаках.